<center><img src="../../../fig/ICHEC_Logo.jpg" alt="Drawing" style="width: 500px;"/>

# Exercise 1 Solutions 
**make sure you have environment 'numba' displayed in the top right corner!**

- Improve the timing for computing this mandel brot set by njiting one of the functions
- submit to Kay using a job submission script
- Calculate the speed up factor

In [1]:
%%writefile mandel_exercise.py

import numpy as np
import math
import time
import numba
from numba import jit, njit


import matplotlib.pyplot as plt
# %matplotlib inline

mandel_timings = []

def plot_mandel(mandel):
    fig=plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.imshow(mandel, cmap='gnuplot')
    plt.savefig('mandel.png')
    
def kernel(zr, zi, cr, ci, radius, num_iters):
    count = 0
    while ((zr*zr + zi*zi) < (radius*radius)) and count < num_iters:
        zr, zi = zr * zr - zi * zi + cr, 2 * zr * zi + ci
        count += 1
    return count

def compute_mandel_py(cr, ci, N, bound, radius=1000.):
    t0 = time.time()
    mandel = np.empty((N, N), dtype=int)
    grid_x = np.linspace(-bound, bound, N)

    for i, x in enumerate(grid_x):
        for j, y in enumerate(grid_x):
            mandel[i,j] = kernel(x, y, cr, ci, radius, N)
    return mandel, time.time() - t0

def python_run():
    kwargs = dict(cr=0.3852, ci=-0.2026,
              N=400,
              bound=1.2)
    print("Using pure Python")
    mandel_func = compute_mandel_py       
    mandel_set = mandel_set, runtime = mandel_func(**kwargs)
    
    print("Mandelbrot set generated in {} seconds".format(runtime))
    #plot_mandel(mandel_set)
    mandel_timings.append(runtime)

    
python_run()

kernel_njit = njit(kernel)

# copy and paste the compute_mandle_py function & rename it 
# change the kernel function used within it
def compute_mandel_njit(cr, ci, N, bound, radius=1000.):
    t0 = time.time()
    mandel = np.empty((N, N), dtype=int)
    grid_x = np.linspace(-bound, bound, N)

    for i, x in enumerate(grid_x):
        for j, y in enumerate(grid_x):
            mandel[i,j] = kernel_njit(x, y, cr, ci, radius, N)
    return mandel, time.time() - t0

# copy and paste the run function & rename it
# use the new compute mandel function within it 
def njit_run():
    kwargs = dict(cr=0.3852, ci=-0.2026,
              N=400,
              bound=1.2)
    print("Using njit kernel")
    mandel_func = compute_mandel_njit      
    mandel_set = mandel_set, runtime = mandel_func(**kwargs)
    
    print("Mandelbrot set generated in {} seconds".format(runtime))
    plot_mandel(mandel_set)
    mandel_timings.append(runtime)


## run it twice, first one includes compilation time
njit_run()
njit_run()

print(mandel_timings)

# what percentage did it speed up by?
print("speedup factor using njit on the kernel")
print(mandel_timings[0]/mandel_timings[2])

Overwriting mandel_exercise.py


In [2]:
%%writefile mandel_job.sh
#!/bin/bash
#SBATCH --nodes=1
#SBATCH --time=00:10:00
#SBATCH -A course
#SBATCH --job-name=mandel
#SBATCH -p CourseDevQ
#SBATCH --reservation=CourseMay


module purge
module load conda
module list

source activate numba

cd $SLURM_SUBMIT_DIR

python mandel_exercise.py

exit 0

Writing job.sh


# Exercise 2 - Solutions
Try out `@njit`, eager compilation, cache and fastmath on the Python function below and compare timings.

In [4]:
%%writefile montecarlo_exercise.py

import numpy as np
import math
import time
import numba
from numba import jit, njit

def pi_montecarlo_python(n):
    in_circle = 0
    for i in range(n):
        x, y = np.random.random(), np.random.random()
        if x ** 2 + y ** 2 <= 1.0:
            in_circle += 1
        
    return 4.0 * in_circle / n


pi_montecarlo_python_njit = njit(pi_montecarlo_python)

pi_montecarlo_python_eag = njit(['float32(int32)','float64(int64)'])(pi_montecarlo_python)

pi_montecarlo_python_fm = njit(fastmath=True)(pi_montecarlo_python)

pi_montecarlo_python_cache = njit(cache=True)(pi_montecarlo_python)

pi_montecarlo_python_ALL = njit(['float32(int32)','float64(int64)'],fastmath=True,cache=True)(pi_montecarlo_python)


n = 10000000

print('python njit including compilation')
t0 = time.time()
pi_montecarlo_python_njit(n)
print(time.time()-t0)
print("")

print('python njit')
t0 = time.time()
pi_montecarlo_python_njit(n)
print(time.time()-t0)
print("")

print('python Eager including compilation')
t0 = time.time()
pi_montecarlo_python_eag(n)
print(time.time()-t0)
print("")

print('python Eager')
t0 = time.time()
pi_montecarlo_python_eag(n)
print(time.time()-t0)
print("")

print('python fastmath including compilation')
t0 = time.time()
pi_montecarlo_python_fm(n)
print(time.time()-t0)
print("")

print('python fastmath')
t0 = time.time()
pi_montecarlo_python_fm(n)
print(time.time()-t0)
print("")

print('python cached including compilation')
t0 = time.time()
pi_montecarlo_python_cache(n)
print(time.time()-t0)
print("")

print('python cached')
t0 = time.time()
pi_montecarlo_python_cache(n)
print(time.time()-t0)
print("")

print('python ALL including compilation')
t0 = time.time()
pi_montecarlo_python_ALL(n)
print(time.time()-t0)
print("")

print('python ALL')
t0 = time.time()
pi_montecarlo_python_ALL(n)
print(time.time()-t0)




Writing mandel_exercise.py


In [3]:
%%writefile montecarlo_job.sh
#!/bin/bash
#SBATCH --nodes=1
#SBATCH --time=00:10:00
#SBATCH -A course
#SBATCH --job-name=montecarlo
#SBATCH -p CourseDevQ
#SBATCH --reservation=CourseMay


module purge
module load conda
module list

source activate numba

cd $SLURM_SUBMIT_DIR

python montecarlo_exercise.py

exit 0

Writing mandel_job.sh


# Exercise 3 - Solutions
- See is_prime function below.
- njit it, vectorize it, and vectorize it with target set to parallel.
- Compare all 3 timings

In [1]:
%%writefile prime_exercise.py

import numpy as np
import math
import time
import numba
from numba import jit, njit

def is_prime(n):
    if n <= 1:
        raise ArithmeticError('n <= 1')
    if n == 2 or n == 3:
        return True
    elif n % 2 == 0:
        return False
    else:
        n_sqrt = math.ceil(math.sqrt(n))
        for i in range(3, n_sqrt):
            if n % i == 0:
                return False

    return True

numbers = np.random.randint(2, 100000, dtype=np.int64, size=1000000)

is_prime_njit = njit(is_prime)
is_prime_v = numba.vectorize('boolean(int64)')(is_prime)
is_prime_vp = numba.vectorize('boolean(int64)', target='parallel')(is_prime)


print("Pure Python")
t0 = time.time()
p1 = [is_prime(x) for x in numbers]
print(time.time()-t0)
print("")

print("njit")
t0 = time.time()
p2 = [is_prime_njit(x) for x in numbers]
print(time.time()-t0)

t0 = time.time()
p2 = [is_prime_njit(x) for x in numbers]
print(time.time()-t0)
print("")

print("vectorized")
t0 = time.time()
p3 = is_prime_v(numbers)
print(time.time()-t0)
print("")

print("vectorized and parallel")
t0 = time.time()
p3 = is_prime_vp(numbers)
print(time.time()-t0)

Writing prime_exercise.py


In [3]:
%%writefile prime_job.sh
#!/bin/bash
#SBATCH --nodes=1
#SBATCH --time=00:10:00
#SBATCH -A course
#SBATCH --job-name=prime
#SBATCH -p CourseDevQ
#SBATCH --reservation=CourseMay


module purge
module load conda
module list

source activate numba

cd $SLURM_SUBMIT_DIR

python prime_exercise.py

exit 0

Writing heat_job.sh


# BIG EXERCISE - Solutions

**Using what you've learnt play around with the functions that implement the heat equation.**

### Answer the following questions (can use bottle_large.dat - time differences are more apparent)

1) Which version of the evolve function respond better to optimisation by Numba, the Python or the NumPy one?

2) Try eager compilation on the better function - record time
    - Hint: input type for field is `float64[:,:]`
    
3) Try things like `parallel=True`, `prange`, `fastmath=True`. What combination gave the best optimisation?

4) Which loop is better to have the `prange` on?

5) `njit` the iterate function (solve the error first [hint: comment out python object part], does it speed up? 

6) notice anything wrong when you parallelise the iterate function?

## Solutions
1) evolve python is faster then the numpy one

2) `2.382380247116089` for eager compilation 

3) all 3 `eager parallel and fastmath` `1.4096732139587402`

4) prange is only faster on outer loop.

5) No, njit on iterate function slowed it down

6) look at images - not smooth. Makes sense, iterate relies on the previous iteration, parallelising them puts them out of sync / at the same time 

- Timings given above are from using bottle_large.dat - larger time differences

```
Time for njit evolve numpy 
6.7305943965911865

Time for njit evolve python 
2.504594087600708

Time for njit evolve eager 
2.39389705657959

Time for njit evolve eager parallel 
1.4154531955718994

Time for njit evolve parallel 
2.060504674911499

Time for njit evolve parallel and fastmath 
1.9520213603973389

Time for njit evolve eager parallel and fastmath 
1.3853354454040527

Time for njit iterate on njit  evolve eager parallel and fastmath 
1.6199710369110107

Time for njit parallel iterate on njit  evolve eager parallel and fastmath 
1.9841651916503906

```

In [8]:
%%writefile heat_exercise.py
import matplotlib.pyplot as plt
import numpy as np
import time

import numba
from numba import jit, njit,prange


# Set the colormap
plt.rcParams['image.cmap'] = 'BrBG'

# Basic parameters
a = 0.5                # Diffusion constant
timesteps = 200        # Number of time-steps to evolve system
image_interval = 4000  # Write frequency for png files

# Grid spacings
dx = 0.01
dy = 0.01
dx2 = dx**2
dy2 = dy**2

# For stability, this is the largest interval possible
# for the size of the time-step:
dt = dx2*dy2 / ( 2*a*(dx2+dy2) )


def evolve_python(u, u_previous, a, dt, dx2, dy2):
    
    n = u.shape[0]
    m = u.shape[1]

    for i in prange(1, n-1):
        for j in range(1, m-1):
            u[i, j] = u_previous[i, j] + a * dt * ( \
             (u_previous[i+1, j] - 2*u_previous[i, j] + \
              u_previous[i-1, j]) /dx2 + \
             (u_previous[i, j+1] - 2*u_previous[i, j] + \
                 u_previous[i, j-1]) /dy2 )
    u_previous[:] = u[:]
    

def evolve_numpy(u, u_previous, a, dt, dx2, dy2):

    del_sqrd_u =((u_previous[:-2, 1:-1] - 2*u_previous[1:-1, 1:-1] + u_previous[2:, 1:-1]) / 
                 dx2 + (u_previous[1:-1, :-2] - 2*u_previous[1:-1, 1:-1] + u_previous[1:-1, 2:]) / dy2) 
    
    u[1:-1,1:-1] = u_previous[1:-1,1:-1] + dt * a *  del_sqrd_u

    u_previous[:,:] = u[:,:]

def initial_fields(filename):
    field = np.loadtxt(filename)
    field0 = np.copy(field)
    return field, field0


def write_field(field, step):
    plt.gca().clear()
    #plt.hold(False)
    plt.imshow(field)
    plt.axis('off')
    plt.savefig('heat_{0:03d}.png'.format(step))


evolve_python_njit = njit(evolve_python)
evolve_numpy_njit = njit(evolve_numpy)

# python version is faster then numpy version 

evolve_eag=njit('(float64[:,:],float64[:,:],float64,float64,float64,float64)')(evolve_python)
evolve_eag_par=njit('(float64[:,:],float64[:,:],float64,float64,float64,float64)',parallel=True)(evolve_python)

evolve_par = njit(parallel=True)(evolve_python)
evolve_parfm = njit(parallel=True,fastmath=True)(evolve_python)
evolve_eagparfm = njit('(float64[:,:],float64[:,:],float64,float64,float64,float64)', parallel=True,fastmath=True)(evolve_python)
    
    

    
def iterate1(field, field0, timesteps, image_interval): 
    
    for i in range(1,timesteps+1):
        evolve_numpy_njit(field, field0, a, dt, dx2, dy2)
        if i % image_interval == 0:
            write_field(field, i)
            
def iterate2(field, field0, timesteps, image_interval): 
    
    for i in range(1,timesteps+1):
        evolve_python_njit(field, field0, a, dt, dx2, dy2)
        if i % image_interval == 0:
            write_field(field, i)
            
def iterate3(field, field0, timesteps, image_interval): 
    
    for i in range(1,timesteps+1):
        evolve_eag(field, field0, a, dt, dx2, dy2)
        if i % image_interval == 0:
            write_field(field, i)
            
def iterate4(field, field0, timesteps, image_interval): 
    
    for i in range(1,timesteps+1):
        evolve_eag_par(field, field0, a, dt, dx2, dy2)
        if i % image_interval == 0:
            write_field(field, i)

def iterate5(field, field0, timesteps, image_interval): 
    
    for i in range(1,timesteps+1):
        evolve_par(field, field0, a, dt, dx2, dy2)
        if i % image_interval == 0:
            write_field(field, i)
            
def iterate6(field, field0, timesteps, image_interval): 
    
    for i in range(1,timesteps+1):
        evolve_parfm(field, field0, a, dt, dx2, dy2)
        if i % image_interval == 0:
            write_field(field, i)
            
def iterate7(field, field0, timesteps, image_interval): 
    
    for i in prange(1,timesteps+1):
        evolve_eagparfm(field, field0, a, dt, dx2, dy2)
#         if i % image_interval == 0:
#             write_field(field, i)
            
def main1():
    field, field0 = initial_fields('bottle_large.dat')

    write_field(field, 0)
    
    iterate1(field, field0, timesteps, image_interval)
    
    write_field(field, timesteps)

def main2():
    field, field0 = initial_fields('bottle_large.dat')

    write_field(field, 0)
    
    iterate2(field, field0, timesteps, image_interval)
    
    write_field(field, timesteps)
    
def main3():
    field, field0 = initial_fields('bottle_large.dat')

    write_field(field, 0)
    
    iterate3(field, field0, timesteps, image_interval)
    
    write_field(field, timesteps)

def main4():
    field, field0 = initial_fields('bottle_large.dat')

    write_field(field, 0)
    
    iterate4(field, field0, timesteps, image_interval)
    
    write_field(field, timesteps)
    
def main5():
    field, field0 = initial_fields('bottle_large.dat')

    write_field(field, 0)
    
    iterate5(field, field0, timesteps, image_interval)
    
    write_field(field, timesteps)

def main6():
    field, field0 = initial_fields('bottle_large.dat')

    write_field(field, 0)
    
    iterate6(field, field0, timesteps, image_interval)
    
    write_field(field, timesteps)

def main7():
    field, field0 = initial_fields('bottle_large.dat')

    write_field(field, 0)
    
    iterate7(field, field0, timesteps, image_interval)
    
    write_field(field, timesteps)

njit_iterate = njit(iterate7)
njit_iterateP = njit(parallel=True)(iterate7)

def main8():
    field, field0 = initial_fields('bottle_large.dat')

    write_field(field, 0)
    
    njit_iterate(field, field0, timesteps, image_interval)
    
    write_field(field, timesteps)
    

def main9():
    field, field0 = initial_fields('bottle_large.dat')

    write_field(field, 0)
    
    njit_iterateP(field, field0, timesteps, image_interval)
    
    write_field(field, timesteps)

print("Time for njit evolve numpy ")
t0 = time.time()
main1()
print(time.time()-t0)
print("")

print("Time for njit evolve python ")
t0 = time.time()
main2()
print(time.time()-t0)
print("")

print("Time for njit evolve eager ")
t0 = time.time()
main3()
print(time.time()-t0)
print("")

print("Time for njit evolve eager parallel ")
t0 = time.time()
main4()
print(time.time()-t0)
print("")

print("Time for njit evolve parallel ")
t0 = time.time()
main5()
print(time.time()-t0)
print("")

print("Time for njit evolve parallel and fastmath ")
t0 = time.time()
main6()
print(time.time()-t0)
print("")

print("Time for njit evolve eager parallel and fastmath ")
t0 = time.time()
main7()
print(time.time()-t0)
print("")

print("Time for njit iterate on njit  evolve eager parallel and fastmath ")
t0 = time.time()
main8()
print(time.time()-t0)
print("")

print("Time for njit parallel iterate on njit  evolve eager parallel and fastmath ")
t0 = time.time()
main9()
print(time.time()-t0)
print("")




Writing heat_exercise.py


In [4]:
%%writefile heat_job.sh
#!/bin/bash
#SBATCH --nodes=1
#SBATCH --time=00:10:00
#SBATCH -A course
#SBATCH --job-name=heat
#SBATCH -p CourseDevQ
#SBATCH --reservation=CourseMay


module purge
module load conda
module list

source activate numba

cd $SLURM_SUBMIT_DIR

python heat_exercise.py

exit 0

Writing heat_job.sh
